In [30]:
from plotly.offline import iplot, init_notebook_mode
import json
import random
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import glob
import datetime
import operator
import random

init_notebook_mode(connected=True)

def load_metadata (datadir):
    filenames = glob.glob(datadir + '/*', recursive=False)
    filenames = [file for file in filenames if not file.endswith("png")]
    
    meta = list()
    for file in filenames:
        f = open(file)
        
        m = json.loads(f.read().replace("'", '"'))
        m["gps"] = get_lat_long(0)
        m["category"] = random.randint(0, 1)
        m["timestamp"] = datetime.datetime.strptime(m["timestamp"], '%Y-%m-%d %H:%M:%S.%f')
        m["filename"] = file.split("\\")[-1]
        meta.append(m)
        f.close()
        
    return meta

# Latitude, longitude
def get_lat_long (data):
    return random.uniform(33.824, 36.35), random.uniform(-84, -75)

def get_timestamp (data):
    return random.randint(1, 366)

metadata = load_metadata("../../data/metadata")

metadata = sorted(metadata, key=operator.itemgetter("timestamp"))

# Format dates
for m in metadata:
     m['timestamp'] = m["timestamp"].date().strftime('%m/%d/%Y')

# GPS array for convenience
gps = [m["gps"] for m in metadata]

# Dates array for plotly slider
dates = [m["timestamp"] for m in metadata]
seen = set()
dates = [d for d in dates if d not in seen and not seen.add(d)]

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout'] = {
    'hovermode': 'closest',
    'geo': {
        'scope': 'usa',
        'projection': dict(type = 'albers usa'),
        'showland': True,
        'landcolor': "rgb(250, 250, 250)",
        'subunitcolor': "rgb(217, 217, 217)",
        'countrycolor': "rgb(217, 217, 217)",
        'countrywidth': 0.5,
        'subunitwidth': 0.5    
    },
    'updatemenus': [{
        'buttons': [
            {
                'args': [None, {
                         'frame': {
                             'duration': 500, 
                             'redraw': False
                         },
                         'fromcurrent': True, 
                         'transition': {
                             'duration': 300, 
                             'easing': 'quadratic-in-out'
                         }
                     }
                ],
                'label': 'Play',
                'method': 'animate'
            }, 
            {
                'args': [[None], {
                        'frame': {
                            'duration': 0, 
                            'redraw': False
                        }, 
                        'mode': 'immediate',
                        'transition': {
                            'duration': 0
                        }
                    }
                ],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
}

figure['data'] = [
    go.Scattergeo(
        locationmode= 'USA-states',
        lon= [m['gps'][1] for m in metadata if m['timestamp'] == dates[0]],
        lat= [m['gps'][0] for m in metadata if m['timestamp'] == dates[0]],
        text = [m['filename'] for m in metadata if m['timestamp'] == dates[0]],
        mode= 'markers'
    )]
    
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 18},
        'prefix': 'Date: ',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

for date in dates:
    calls = [call for call in metadata if call['timestamp'] == date]
    for call in calls:
        if call['timestamp'] == date:
            if call['category'] == 1:
                frame = {
                    'name': str(date),
                    'data': 
                        [go.Scattergeo(
                            locationmode = 'USA-states',
                            lon = [call['gps'][1] for call in calls], 
                            lat = [call["gps"][0]for call in calls], 
                            text = call['filename'],
                            mode = 'markers',
                            marker = {
                                'color': 'red'
                            }
                    )]
                }
        
            else:
                frame = {
                    'name': str(date),
                    'data': 
                        [go.Scattergeo(
                            locationmode = 'USA-states',
                            lon = [call["gps"][1] for call in calls], 
                            lat = [call["gps"][0] for call in calls], 
                            text = call['filename'],
                            mode = 'markers',
                            marker = {
                                'color': 'blue'
                            }
                    )]
                }
        
    figure['frames'].append(frame)
    
    slider_step = {
        'args': [
            [date],
            {
                'frame': {
                    'duration': 300, 
                    'redraw': False
                },
                'mode': 'immediate',
                'transition': {
                    'duration': 300
                }
            }
         ],
         'label': date,
         'method': 'animate'
    }
    
    sliders_dict['steps'].append(slider_step)
    
    #break

figure['layout']['sliders'] = [sliders_dict]
iplot(figure, filename="map_test")

In [202]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import numpy as np

init_notebook_mode(connected=True)

t=np.linspace(-1,1,100)
x=t+t**2
y=t-t**2
xm=np.min(x)-1.5
xM=np.max(x)+1.5
ym=np.min(y)-1.5
yM=np.max(y)+1.5
N=50
s=np.linspace(-1,1,N)
xx=s+s**2
yy=s-s**2

data=[dict(x=x, y=y, 
           mode='lines', 
           line=dict(width=2, color='blue')
          ),
      dict(x=x, y=y, 
           mode='lines', 
           line=dict(width=2, color='blue')
          )
    ]

layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
            yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
            title='Kinematic Generation of a Planar Curve', hovermode='closest',
            updatemenus= [{'type': 'buttons',
                           'buttons': [{'label': 'Play',
                                        'method': 'animate',
                                        'args': [None]}]}])

frames=[dict(data=[dict(x=[xx[k]], 
                        y=[yy[k]], 
                        mode='markers', 
                        marker=dict(color='red', size=10)
                        )
                  ]) for k in range(N)]    

print (frames)

figure1=dict(data=data, layout=layout, frames=frames)          
iplot(figure1)

#py.icreate_animations(figure, filename='animation_test'+str(time.time()))

[{'data': [{'x': [0.0], 'y': [-2.0], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.03915035401915867], 'y': [-1.8792169929196167], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.07496876301541022], 'y': [-1.7617659308621407], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.10745522698875476], 'y': [-1.6476468138275717], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.13660974593919195], 'y': [-1.5368596418159102], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.16243231986672224], 'y': [-1.4294044148271552], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.1849229487713453], 'y': [-1.3252811328613077], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.20408163265306123], 'y': [-1.2244897959183674], 'mode': 'markers', 'marker': {'color': 'red', 'size': 

In [70]:
figure['data'] = {
    'type': 'scattergeo',
    'locationmode': 'USA-states',
    'lon': [m['gps'][1] for m in metadata if m['timestamp'].date().strftime('%m/%d/%Y') == timestamp[0]],
    'lat': [m['gps'][0] for m in metadata if m['timestamp'].date().strftime('%m/%d/%Y') == timestamp[0]],
    'mode': 'markers'
}

In [250]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

import pandas as pd

init_notebook_mode(connected=True)

url = 'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
dataset = pd.read_csv(url)

years = ['1952', '1962', '1967', '1972', '1977', '1982', '1987', '1992', '1997', '2002', '2007']
# make list of continents
continents = []
for continent in dataset['continent']:
    if continent not in continents:
        continents.append(continent)
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [30, 85], 'title': 'Life Expectancy'}
figure['layout']['yaxis'] = {'title': 'GDP per Capita', 'type': 'log'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1952',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = 1952
for continent in continents:
    dataset_by_year = dataset[dataset['year'] == year]
    dataset_by_year_and_cont = dataset_by_year[dataset_by_year['continent'] == continent]

    data_dict = {
        'x': list(dataset_by_year_and_cont['lifeExp']),
        'y': list(dataset_by_year_and_cont['gdpPercap']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cont['country']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'size': list(dataset_by_year_and_cont['pop'])
        },
        'name': continent
    }
    figure['data'].append(data_dict)
    
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        dataset_by_year = dataset[dataset['year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['continent'] == continent]

        data_dict = {
            'x': list(dataset_by_year_and_cont['lifeExp']),
            'y': list(dataset_by_year_and_cont['gdpPercap']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cont['country']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,
                'size': list(dataset_by_year_and_cont['pop'])
            },
            'name': continent
        }
        print('\n', data_dict)
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

iplot(figure)


 {'x': [28.801, 50.93899999999999, 37.484, 39.417, 44.0, 60.96, 37.373000000000005, 37.468, 44.869, 45.32, 65.39, 63.03, 43.158, 50.056000000000004, 47.453, 55.565, 55.928000000000004, 48.463, 42.244, 36.319, 36.157, 37.578, 43.43600000000001, 47.751999999999995, 39.875, 60.396, 57.593, 45.883, 58.5, 50.848, 40.412, 43.16, 32.548], 'y': [779.4453145, 9867.084765000001, 684.2441716, 368.46928560000003, 400.44861069999996, 3054.421209, 546.5657493, 749.6816546, 3035.326002, 4129.766056, 4086.522128, 3216.956347, 1546.907807, 1088.277758, 1030.592226, 108382.3529, 4834.804067, 1831.132894, 786.5668575, 331.0, 545.8657228999999, 1828.230307, 684.5971437999999, 1272.880995, 6459.5548229999995, 2315.138227, 1083.53203, 1643.485354, 1206.947913, 757.7974177, 605.0664917, 1515.5923289999998, 781.7175761], 'mode': 'markers', 'text': ['Afghanistan', 'Bahrain', 'Bangladesh', 'Cambodia', 'China', 'Hong Kong, China', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Korea, Dem. R